### Batch Train Word2Vec Model

In [2]:
import sys
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.word2vec import Word2Vec
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import *
import string
import os
from bs4 import BeautifulSoup
import re
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from pathlib import Path

In [3]:
# german stopwords
de_stop_words = stopwords.words('german')
de_stop_words_cap = [x.capitalize() for x in de_stop_words]

# creating a stopword dictionary for simplifying replacement
de_stop_words_dict = {de_stop_words_cap[i]: de_stop_words[i] for i in range(len(de_stop_words_cap))} 

In [4]:
# function for replacing elements in a list matching the given dictionary keys
def replace_matched_items(word_list, dictionary):
    for lst in word_list:
        for ind, item in enumerate(lst):
            lst[ind] = dictionary.get(item, item)

In [14]:
train = []
dir_list = os.listdir(os.getcwd() + "\\extracted\\")
file_list = []
l = 0
file = 0
ticker = 0
max_batch = 2

while ticker < 58:
    
    while l < max_batch:
        for filename in os.listdir(os.getcwd() + "\\extracted\\" + dir_list[file]):
            with open(os.path.join(os.getcwd() + "\\extracted\\" + dir_list[file] + "\\" + filename), 'rt', encoding="utf-8") as f:
                train.append(f.read())
        file += 1
        l += 1
        
    clean_train = [x.encode('utf-8').decode('utf-8') for x in train]

    # cleaning
    
    # removing double space, chars and line breaks
    clean_train = [((x.replace("\xa0", " ")).replace("\n", " ")).replace("  ", " ") for x in clean_train] 

    # remove <doc> tag, link and article id in beginning
    clean_train = [re.sub(r'<.+?> ', '', x) for x in clean_train]
  
    # remove digits, and the following words f.e. 1. August 2020 - 1. and 2020 are removed
    clean_train = [re.sub("\S*\d\S*", '', x).strip() for x in clean_train]

    # tokenize in list of articles with sentences as list
    clean_train = [sent_tokenize(x) for x in clean_train]

    # flatten list = one big list with multiple sentences - no article recognition anymore - usefull (?)
    clean_train = [item for sublist in clean_train for item in sublist]

    # remove punctuation
    clean_train = [re.sub(r'[^\w\s]', '', x).strip() for x in clean_train]

    # tokenize all words in sentences = on bis list, with list of articles with tokenized words
    clean_train = [word_tokenize(x) for x in clean_train]    

    replace_matched_items(clean_train, de_stop_words_dict)

    #training
    
    if ticker < max_batch:
        model = Word2Vec(clean_train)
        model.save("w2v_model.bin")
        del model
        print("Model created at ticker ", ticker) 
        train = []
        ticker += max_batch
        l = 0
    else:
        model = gensim.models.Word2Vec.load("w2v_model.bin")
        model.train(clean_train, total_examples=(len(clean_train)), epochs=model.epochs)
        model.save("w2v_model.bin")
        del model
        print("Model updated at ticker ", ticker) 
        train = []
        ticker += max_batch
        l = 0

KeyboardInterrupt: 